## Classification

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv("diabetes_health.csv")

# Display the first 2 rows of the dataset to get a quick look at the data
data.head(2)

# Generate summary statistics for numerical columns (count, mean, std, min, quartiles, max)
data.describe()

# Print concise information about the DataFrame: column names, non-null counts, and data types
data.info()

# Count the number of missing (NaN) values in each column
data.isnull().sum()

# Display the list of all column names in the dataset
data.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126839 entries, 0 to 126838
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype
---  ------                --------------   -----
 0   HeartDiseaseorAttack  126839 non-null  int64
 1   HighBP                126839 non-null  int64
 2   HighChol              126839 non-null  int64
 3   CholCheck             126839 non-null  int64
 4   BMI                   126839 non-null  int64
 5   Smoker                126839 non-null  int64
 6   Stroke                126839 non-null  int64
 7   Diabetes              126839 non-null  int64
 8   PhysActivity          126839 non-null  int64
 9   Fruits                126839 non-null  int64
 10  Veggies               126839 non-null  int64
 11  HvyAlcoholConsump     126839 non-null  int64
 12  AnyHealthcare         126839 non-null  int64
 13  NoDocbcCost           126839 non-null  int64
 14  GenHlth               126839 non-null  int64
 15  MentHlth              126839 non-n

Index(['HeartDiseaseorAttack', 'HighBP', 'HighChol', 'CholCheck', 'BMI',
       'Smoker', 'Stroke', 'Diabetes', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

In [4]:
data.isnull().sum()

HeartDiseaseorAttack    0
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
Diabetes                0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
dtype: int64

In [5]:
import json, math
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_recall_curve,
    confusion_matrix, precision_score, recall_score, f1_score
)

# Helper
def to_py(o):
    if isinstance(o, np.generic):
        return o.item()
    if isinstance(o, np.ndarray):
        return o.tolist()
    if isinstance(o, dict):
        return {str(k): to_py(v) for k, v in o.items()}
    if isinstance(o, (list, tuple)):
        return [to_py(v) for v in o]
    return o

# 1. Load data
df = pd.read_csv("diabetes_health.csv")
target_col = "Diabetes"
y = (df[target_col] > 0).astype(int).values
X = df.drop(columns=[target_col])

# Split categorical vs numeric
categorical_cols, numeric_cols = [], []
for c in X.columns:
    if X[c].dtype == "object":
        categorical_cols.append(c)
    elif pd.api.types.is_integer_dtype(X[c]):
        (categorical_cols if X[c].nunique() <= 10 else numeric_cols).append(c)
    else:
        numeric_cols.append(c)

# 2. Train/test split
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. Preprocessing
numeric_transformer = Pipeline([("imputer", SimpleImputer(strategy="median"))])
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])

# 4. Decision Tree tuned for recall
dt = DecisionTreeClassifier(class_weight='balanced', random_state=42)
pipe = Pipeline([
    ("pre", preprocessor),
    ("dt", dt)
])

param_grid = {
    "dt__max_depth": [3, 4, 5, 6, None],
    "dt__min_samples_leaf": [1, 2, 5, 10]
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# >>> KEY CHANGE: use recall as primary scoring metric
grid = GridSearchCV(
    pipe, param_grid, cv=cv,
    scoring="recall", n_jobs=-1
)
grid.fit(X_trainval, y_trainval)

best_pipe = grid.best_estimator_
best_params = grid.best_params_

# 5. Threshold selection — maximize recall subject to F1 floor (optional)
val_proba = best_pipe.predict_proba(X_trainval)[:, 1]
prec, rec, thr = precision_recall_curve(y_trainval, val_proba)

# Choose threshold that achieves max recall with acceptable precision
target_min_precision = 0.3  # adjust based on domain
mask = prec >= target_min_precision
if mask.any():
    chosen_idx = np.argmax(rec[mask])  # maximize recall
    chosen_threshold = thr[np.where(mask)[0][chosen_idx]]
else:
    chosen_threshold = 0.3  # fallback default
chosen_threshold = float(chosen_threshold)

# 6. Evaluate
test_proba = best_pipe.predict_proba(X_test)[:, 1]
test_pred = (test_proba >= chosen_threshold).astype(int)

metrics = {
    "roc_auc": float(roc_auc_score(y_test, test_proba)),
    "pr_auc": float(average_precision_score(y_test, test_proba)),
    "precision": float(precision_score(y_test, test_pred)),
    "recall": float(recall_score(y_test, test_pred)),
    "f1": float(f1_score(y_test, test_pred))
}
tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
metrics["specificity"] = float(tn / (tn + fp + 1e-12))
metrics["confusion_matrix"] = {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)}

# 7. Export artifacts (same as before)
preprocessor.fit(X_trainval)
num_imputer = preprocessor.named_transformers_["num"].named_steps["imputer"]
num_impute_values = {c: float(v) for c, v in zip(numeric_cols, num_imputer.statistics_.tolist())}
num_ranges = {c: {"min": float(X_trainval[c].min()), "max": float(X_trainval[c].max())} for c in numeric_cols}

onehot = preprocessor.named_transformers_["cat"].named_steps["onehot"]
cat_vocabs = {}
for c, cats in zip(categorical_cols, onehot.categories_):
    cat_vocabs[c] = [None if (isinstance(v, float) and math.isnan(v)) else v for v in cats.tolist()]

final_feature_order = []
for col in numeric_cols:
    final_feature_order.append({"source": col, "kind": "numeric"})
for col, cats in cat_vocabs.items():
    for cat in cats:
        final_feature_order.append({"source": col, "kind": "onehot", "category": cat})

preproc_manifest = {
    "numeric": numeric_cols,
    "categorical": categorical_cols,
    "numeric_imputation": num_impute_values,
    "numeric_ranges_train": num_ranges,
    "categorical_vocabulary": cat_vocabs,
    "final_feature_order": final_feature_order,
    "preproc_version": "v1_recall",
    "trained_on": datetime.utcnow().strftime("%Y-%m-%d")
}

# 8. Export tree
def export_tree(tree_estimator):
    tree = tree_estimator.tree_
    return {
        "nodes": [
            {
                "feature_index": int(tree.feature[i]),
                "threshold": float(tree.threshold[i]),
                "left": int(tree.children_left[i]),
                "right": int(tree.children_right[i]),
                "value": [float(x) for x in tree.value[i][0].tolist()],
                "is_leaf": bool(tree.children_left[i] == -1 and tree.children_right[i] == -1),
            }
            for i in range(tree.node_count)
        ]
    }

dt_est = best_pipe.named_steps["dt"]
dt_export = {
    "params": best_params,
    "threshold": chosen_threshold,
    "metrics": metrics,
    "model_version": "v1_recall",
    "trained_on": datetime.utcnow().strftime("%Y-%m-%d"),
    "tree": export_tree(dt_est)
}

# 9. Model Card
model_card = f"""# Diabetes Decision Tree Model (Recall-Prioritized)

**Model version:** v1  
**Training date:** {datetime.utcnow().strftime("%Y-%m-%d")}  

### Best Params
{best_params}

### Metrics (Test Set)
{json.dumps(metrics, indent=2)}

### Objective
Optimized for *recall*, minimizing false negatives (e.g., catching as many positive cases as possible).

### Notes
Educational demo only — not medical advice.
"""

# 10. Golden Examples
golden = []
for i in range(min(10, len(X_test))):
    row = X_test.iloc[i].to_dict()
    golden.append({
        "input": to_py(row),
        "pred_proba_high": float(test_proba[i]),
        "label": int(test_pred[i]),
        "true": int(y_test[i])
    })

# 10. Save
with open("preproc_v1.json","w") as f: json.dump(to_py(preproc_manifest),f,indent=2)
with open("dt_model_v1.json","w") as f: json.dump(to_py(dt_export),f,indent=2)
with open("model_card_v1.md","w") as f: f.write(model_card)
with open("golden_examples_v1.json","w") as f: json.dump(to_py(golden),f,indent=2)

print("Artifacts written: preproc_v1_recall.json, dt_model_v1_recall.json, model_card_v1_recall.md")

Artifacts written: preproc_v1_recall.json, dt_model_v1_recall.json, model_card_v1_recall.md


In [6]:
import json
import numpy as np

class DecisionTreePredictor:
    def __init__(self, preproc_path="preproc_v1.json", model_path="dt_model_v1.json"):
        """Initialize the predictor with preprocessing and model configs"""
        with open(preproc_path, 'r') as f:
            self.preproc = json.load(f)
        with open(model_path, 'r') as f:
            self.model = json.load(f)
        
        self.tree = self.model['tree']['nodes']
        self.threshold = self.model['threshold']
        
    def get_user_input(self):
        """Collect user input for all features"""
        print("\n=== Heart Disease Risk Assessment ===")
        print("Please enter the following information:\n")
        
        user_data = {}
        
        # Collect numeric features
        for col in self.preproc['numeric']:
            range_info = self.preproc['numeric_ranges_train'][col]
            while True:
                try:
                    value = input(f"{col} (typical range: {range_info['min']:.1f}-{range_info['max']:.1f}): ").strip()
                    if value == "":
                        # Use imputation value if empty
                        user_data[col] = None
                        print(f"  → Using median value: {self.preproc['numeric_imputation'][col]:.1f}")
                        break
                    else:
                        user_data[col] = float(value)
                        break
                except ValueError:
                    print("  Please enter a valid number or press Enter to use default")
        
        # Collect categorical features
        for col in self.preproc['categorical']:
            vocab = self.preproc['categorical_vocabulary'][col]
            print(f"\n{col} options: {', '.join(str(v) for v in vocab if v is not None)}")
            while True:
                value = input(f"{col}: ").strip()
                if value == "":
                    user_data[col] = None
                    print(f"  → Using most frequent value")
                    break
                # Try to convert to appropriate type
                try:
                    if value.isdigit():
                        value = int(value)
                    user_data[col] = value
                    break
                except:
                    user_data[col] = value
                    break
        
        return user_data
    
    def preprocess_input(self, user_data):
        """Apply preprocessing to user input"""
        features = []
        
        # Process numeric features
        for col in self.preproc['numeric']:
            value = user_data.get(col)
            if value is None:
                # Apply imputation
                value = self.preproc['numeric_imputation'][col]
            features.append(value)
        
        # Process categorical features with one-hot encoding
        for col in self.preproc['categorical']:
            value = user_data.get(col)
            vocab = self.preproc['categorical_vocabulary'][col]
            
            # Create one-hot encoding
            for category in vocab:
                if value == category:
                    features.append(1.0)
                else:
                    features.append(0.0)
        
        return np.array(features)
    
    def trace_tree(self, features):
        """Trace through the decision tree and return path"""
        path = []
        node_idx = 0  # Start at root
        
        while True:
            node = self.tree[node_idx]
            
            if node['is_leaf']:
                # Reached a leaf node
                path.append({
                    'node': node_idx,
                    'type': 'leaf',
                    'values': node['value']
                })
                break
            
            # Get feature name for this split
            feature_idx = node['feature_index']
            feature_info = self.preproc['final_feature_order'][feature_idx]
            
            # Make decision
            feature_value = features[feature_idx]
            threshold = node['threshold']
            
            if feature_value <= threshold:
                next_node = node['left']
                direction = 'left'
                condition = f"<= {threshold:.3f}"
            else:
                next_node = node['right']
                direction = 'right'
                condition = f"> {threshold:.3f}"
            
            path.append({
                'node': node_idx,
                'type': 'decision',
                'feature': feature_info,
                'feature_value': feature_value,
                'threshold': threshold,
                'direction': direction,
                'condition': condition
            })
            
            node_idx = next_node
        
        return path
    
    def predict(self, features):
        """Make prediction using the decision tree"""
        # Trace to leaf node
        node_idx = 0
        while not self.tree[node_idx]['is_leaf']:
            node = self.tree[node_idx]
            if features[node['feature_index']] <= node['threshold']:
                node_idx = node['left']
            else:
                node_idx = node['right']
        
        # Get prediction from leaf node
        leaf_values = self.tree[node_idx]['value']
        # Convert to probability (positive class)
        total = sum(leaf_values)
        prob_positive = leaf_values[1] / total if total > 0 else 0
        
        return prob_positive
    
    def display_results(self, user_data, features, path, prob_positive):
        """Display the prediction results and decision path"""
        print("\n" + "="*50)
        print("PREDICTION RESULTS")
        print("="*50)
        
        # Show input summary
        print("\n📊 Input Summary:")
        for col, value in user_data.items():
            if value is not None:
                print(f"  • {col}: {value}")
            else:
                if col in self.preproc['numeric']:
                    print(f"  • {col}: {self.preproc['numeric_imputation'][col]:.1f} (imputed)")
                else:
                    print(f"  • {col}: (imputed)")
        
        # Show decision path
        print("\n🌳 Decision Path:")
        for i, step in enumerate(path):
            if step['type'] == 'decision':
                feature = step['feature']
                if feature['kind'] == 'numeric':
                    feature_name = feature['source']
                else:
                    feature_name = f"{feature['source']}={feature['category']}"
                
                print(f"  Step {i+1}: {feature_name} = {step['feature_value']:.3f} {step['condition']}")
                print(f"          → Go {step['direction']}")
            else:
                print(f"  Step {i+1}: Reached leaf node #{step['node']}")
        
        # Show prediction
        print("\n🎯 Prediction:")
        print(f"  Probability of heart disease: {prob_positive:.1%}")
        
        prediction = 1 if prob_positive >= self.threshold else 0
        risk_level = "HIGH" if prediction == 1 else "LOW"
        
        print(f"  Threshold: {self.threshold:.3f}")
        print(f"  Classification: {risk_level} RISK")
        
        # Show model performance context
        print("\n📈 Model Performance (on test set):")
        metrics = self.model['metrics']
        print(f"  • Precision: {metrics['precision']:.1%}")
        print(f"  • Recall: {metrics['recall']:.1%}")
        print(f"  • F1 Score: {metrics['f1']:.1%}")
        
        print("\n⚠️  DISCLAIMER: This is for educational purposes only.")
        print("    Please consult healthcare professionals for medical advice.")

def main():
    """Main function to run the predictor"""
    try:
        # Initialize predictor
        predictor = DecisionTreePredictor()
        
        while True:
            # Get user input
            user_data = predictor.get_user_input()
            
            # Preprocess input
            features = predictor.preprocess_input(user_data)
            
            # Trace decision path
            path = predictor.trace_tree(features)
            
            # Get prediction
            prob_positive = predictor.predict(features)
            
            # Display results
            predictor.display_results(user_data, features, path, prob_positive)
            
            # Ask if user wants to continue
            print("\n" + "="*50)
            again = input("\nWould you like to assess another case? (yes/no): ").strip().lower()
            if again not in ['yes', 'y']:
                print("\nThank you for using the Heart Disease Risk Assessment tool!")
                break
            print("\n" + "="*50)
            
    except FileNotFoundError as e:
        print(f"Error: Could not find required files. Please ensure 'preproc_v1.json' and 'dt_model_v1.json' exist.")
        print(f"Details: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    print(metrics)
    main()

{'roc_auc': 0.7931898368218473, 'pr_auc': 0.391036285507704, 'precision': 0.3092534914096252, 'recall': 0.7710420841683366, 'f1': 0.44144854786661886, 'specificity': 0.6783776197604791, 'confusion_matrix': {'tn': 14501, 'fp': 6875, 'fn': 914, 'tp': 3078}}

=== Heart Disease Risk Assessment ===
Please enter the following information:



BMI (typical range: 12.0-95.0):  0
MentHlth (typical range: 0.0-30.0):  0
PhysHlth (typical range: 0.0-30.0):  0
Age (typical range: 1.0-13.0):  0



HeartDiseaseorAttack options: 0, 1


HeartDiseaseorAttack:  0



HighBP options: 0, 1


...


Income options: 1, 2, 3, 4, 5, 6, 7, 8


Income:  0



PREDICTION RESULTS

📊 Input Summary:
  • BMI: 0.0
  • MentHlth: 0.0
  • PhysHlth: 0.0
  • Age: 0.0
  • HeartDiseaseorAttack: 0
  • HighBP: 0
  • HighChol: 0
  • CholCheck: 0
  • Smoker: 0
  • Stroke: 0
  • PhysActivity: 0
  • Fruits: 0
  • Veggies: 0
  • HvyAlcoholConsump: 0
  • AnyHealthcare: 0
  • NoDocbcCost: 0
  • GenHlth: 0
  • DiffWalk: 0
  • Sex: 0
  • Education: 0
  • Income: 0

🌳 Decision Path:
  Step 1: HighBP=0 = 1.000 > 0.500
          → Go right
  Step 2: BMI = 0.000 <= 28.500
          → Go left
  Step 3: Age = 0.000 <= 8.500
          → Go left
  Step 4: GenHlth=4 = 0.000 <= 0.500
          → Go left
  Step 5: GenHlth=5 = 0.000 <= 0.500
          → Go left
  Step 6: Reached leaf node #36

🎯 Prediction:
  Probability of heart disease: 10.1%
  Threshold: 0.505
  Classification: LOW RISK

📈 Model Performance (on test set):
  • Precision: 30.9%
  • Recall: 77.1%
  • F1 Score: 44.1%

⚠️  DISCLAIMER: This is for educational purposes only.
    Please consult healthcare profess


Would you like to assess another case? (yes/no):  0



Thank you for using the Heart Disease Risk Assessment tool!


In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1. Load data
df = pd.read_csv("diabetes_health.csv")
target_col = "Diabetes"
y = (df[target_col] > 0).astype(int).values
X = df.drop(columns=[target_col])

# Split categorical vs numeric
categorical_cols, numeric_cols = [], []
for c in X.columns:
    if X[c].dtype == "object":
        categorical_cols.append(c)
    elif pd.api.types.is_integer_dtype(X[c]):
        (categorical_cols if X[c].nunique() <= 10 else numeric_cols).append(c)
    else:
        numeric_cols.append(c)

# 2. Train/test split
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. Preprocessing
numeric_transformer = Pipeline([("imputer", SimpleImputer(strategy="median"))])
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])

# 4. Create KNN pipeline
knn_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("knn", KNeighborsClassifier())
])

# 5. Hyperparameter tuning (optimized for recall)
param_grid = {
    'knn__n_neighbors': [3, 5],
    'knn__weights': ['distance'],
    'knn__metric': ['manhattan'],
    'knn__p': [1, 2],  # Power parameter for minkowski metric
    'knn__algorithm': ['auto', 'ball_tree', 'kd_tree']
}

grid_search = GridSearchCV(
    knn_pipeline,
    param_grid,
    cv=5,
    scoring='recall',  # Optimizing for recall instead of accuracy
    n_jobs=-1,
    verbose=1
)

print("Training KNN model with grid search...")
grid_search.fit(X_trainval, y_trainval)

# 6. Best model
best_knn = grid_search.best_estimator_
print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best cross-validation recall: {grid_search.best_score_:.4f}")

# 7. Evaluate on test set
y_pred = best_knn.predict(X_test)
y_pred_proba = best_knn.predict_proba(X_test)[:, 1]

# Adjust decision threshold to increase recall
# Lower values (0.2, 0.25) = Higher recall (catch more diabetes cases, more false positives)
# Higher values (0.4, 0.5) = Lower recall but higher precision (fewer false positives)
threshold = 0.2  # Adjust this value between 0 and 1
y_pred_adjusted = (y_pred_proba >= threshold).astype(int)

test_accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy (default threshold): {test_accuracy:.4f}")
print("\nClassification Report (default threshold):")
print(classification_report(y_test, y_pred))

print(f"\n--- Adjusted Threshold = {threshold} (Higher Recall) ---")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_adjusted):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_adjusted))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_adjusted))

# 8. Alternative: Simple KNN without tuning
simple_knn = Pipeline([
    ("preprocessor", preprocessor),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])
simple_knn.fit(X_trainval, y_trainval)
simple_pred = simple_knn.predict(X_test)
print(f"\nSimple KNN (k=5) Accuracy: {accuracy_score(y_test, simple_pred):.4f}")

Training KNN model with grid search...
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Best parameters: {'knn__algorithm': 'auto', 'knn__metric': 'manhattan', 'knn__n_neighbors': 3, 'knn__p': 1, 'knn__weights': 'distance'}
Best cross-validation recall: 0.2572

Test Accuracy (default threshold): 0.8187

Classification Report (default threshold):
              precision    recall  f1-score   support

           0       0.87      0.92      0.90     21376
           1       0.39      0.26      0.31      3992

    accuracy                           0.82     25368
   macro avg       0.63      0.59      0.60     25368
weighted avg       0.79      0.82      0.80     25368


--- Adjusted Threshold = 0.2 (Higher Recall) ---
Test Accuracy: 0.7048

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.72      0.80     21376
           1       0.29      0.61      0.39      3992

    accuracy                           0.70     2536

In [42]:
grid_search.best_params_

{'knn__algorithm': 'auto',
 'knn__metric': 'manhattan',
 'knn__n_neighbors': 3,
 'knn__p': 1,
 'knn__weights': 'distance'}

In [54]:
import json, math
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_recall_curve,
    confusion_matrix, precision_score, recall_score, f1_score
)

# Helper
def to_py(o):
    if isinstance(o, np.generic):
        return o.item()
    if isinstance(o, np.ndarray):
        return o.tolist()
    if isinstance(o, dict):
        return {str(k): to_py(v) for k, v in o.items()}
    if isinstance(o, (list, tuple)):
        return [to_py(v) for v in o]
    return o

# 1. Load data
df = pd.read_csv("diabetes_health.csv")
target_col = "Diabetes"
y = (df[target_col] > 0).astype(int).values
X = df.drop(columns=[target_col])

# Split categorical vs numeric
categorical_cols, numeric_cols = [], []
for c in X.columns:
    if X[c].dtype == "object":
        categorical_cols.append(c)
    elif pd.api.types.is_integer_dtype(X[c]):
        (categorical_cols if X[c].nunique() <= 10 else numeric_cols).append(c)
    else:
        numeric_cols.append(c)

# 2. Train/test split
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. Preprocessing (with scaling for KNN)
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])

# 4. KNN with GridSearchCV tuned for recall
knn = KNeighborsClassifier()
pipe = Pipeline([
    ("pre", preprocessor),
    ("knn", knn)
])

param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11],
    "knn__weights": ['distance'],
    "knn__metric": ['euclidean', 'manhattan', 'minkowski'],
    "knn__p": [1, 2],
    "knn__algorithm": ['auto']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV with recall scoring
print("Starting hyperparameter tuning...")
grid = GridSearchCV(
    pipe, param_grid, cv=cv,
    scoring="recall", n_jobs=-1, verbose=1
)
grid.fit(X_trainval, y_trainval)

best_pipe = grid.best_estimator_
best_params = grid.best_params_
print(f"\nBest parameters: {best_params}")
print(f"Best CV recall score: {grid.best_score_:.3f}")

# 5. Threshold selection — maximize recall with precision constraint
val_proba = best_pipe.predict_proba(X_trainval)[:, 1]
prec, rec, thr = precision_recall_curve(y_trainval, val_proba)

# Set minimum precision to 40% to avoid guessing all positive
target_min_precision = 0.40
mask = prec >= target_min_precision
if mask.any():
    chosen_idx = np.argmax(rec[mask])  # maximize recall
    chosen_threshold = thr[np.where(mask)[0][chosen_idx]]
else:
    # Fallback: find threshold closest to 40% precision
    closest_idx = np.argmin(np.abs(prec - target_min_precision))
    chosen_threshold = thr[min(closest_idx, len(thr) - 1)]
chosen_threshold = float(chosen_threshold)

# 6. Evaluate
test_proba = best_pipe.predict_proba(X_test)[:, 1]
test_pred = (test_proba >= chosen_threshold).astype(int)

metrics = {
    "roc_auc": float(roc_auc_score(y_test, test_proba)),
    "pr_auc": float(average_precision_score(y_test, test_proba)),
    "precision": float(precision_score(y_test, test_pred)),
    "recall": float(recall_score(y_test, test_pred)),
    "f1": float(f1_score(y_test, test_pred))
}
tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
metrics["specificity"] = float(tn / (tn + fp + 1e-12))
metrics["confusion_matrix"] = {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)}

# 7. Export preprocessing artifacts
preprocessor.fit(X_trainval)
num_imputer = preprocessor.named_transformers_["num"].named_steps["imputer"]
num_scaler = preprocessor.named_transformers_["num"].named_steps["scaler"]
num_impute_values = {c: float(v) for c, v in zip(numeric_cols, num_imputer.statistics_.tolist())}
num_scale_mean = {c: float(v) for c, v in zip(numeric_cols, num_scaler.mean_.tolist())}
num_scale_std = {c: float(v) for c, v in zip(numeric_cols, num_scaler.scale_.tolist())}
num_ranges = {c: {"min": float(X_trainval[c].min()), "max": float(X_trainval[c].max())} for c in numeric_cols}

onehot = preprocessor.named_transformers_["cat"].named_steps["onehot"]
cat_vocabs = {}
for c, cats in zip(categorical_cols, onehot.categories_):
    cat_vocabs[c] = [None if (isinstance(v, float) and math.isnan(v)) else v for v in cats.tolist()]

final_feature_order = []
for col in numeric_cols:
    final_feature_order.append({"source": col, "kind": "numeric"})
for col, cats in cat_vocabs.items():
    for cat in cats:
        final_feature_order.append({"source": col, "kind": "onehot", "category": cat})

preproc_manifest = {
    "numeric": numeric_cols,
    "categorical": categorical_cols,
    "numeric_imputation": num_impute_values,
    "numeric_scaling_mean": num_scale_mean,
    "numeric_scaling_std": num_scale_std,
    "numeric_ranges_train": num_ranges,
    "categorical_vocabulary": cat_vocabs,
    "final_feature_order": final_feature_order,
    "preproc_version": "v2_knn_recall",
    "trained_on": datetime.utcnow().strftime("%Y-%m-%d")
}

# 8. Export KNN model info
knn_export = {
    "params": best_params,
    "threshold": chosen_threshold,
    "metrics": metrics,
    "model_version": "v2_knn_recall",
    "model_type": "KNeighborsClassifier",
    "trained_on": datetime.utcnow().strftime("%Y-%m-%d"),
    "cv_best_score": float(grid.best_score_),
    "note": "KNN is instance-based and requires the full training set for predictions"
}

# 9. Model Card
model_card = f"""# Diabetes KNN Model (Recall-Prioritized)

**Model version:** v2  
**Model type:** K-Nearest Neighbors  
**Training date:** {datetime.utcnow().strftime("%Y-%m-%d")}  

### Best Params (GridSearchCV)
{json.dumps(best_params, indent=2)}

### Cross-Validation
- Best CV Recall Score: {grid.best_score_:.3f}
- Stratified 5-Fold CV

### Metrics (Test Set)
{json.dumps(metrics, indent=2)}

### Objective
Optimized for *recall* via hyperparameter tuning, minimizing false negatives while maintaining precision above 40%.

### Model Details
- Hyperparameter tuning with GridSearchCV
- Distance weighting for neighbors
- Features scaled using StandardScaler
- Focused on lower k values (3-11 neighbors)

### Notes
Educational demo only — not medical advice.
"""

# 10. Golden Examples
golden = []
for i in range(min(10, len(X_test))):
    row = X_test.iloc[i].to_dict()
    golden.append({
        "input": to_py(row),
        "pred_proba_high": float(test_proba[i]),
        "label": int(test_pred[i]),
        "true": int(y_test[i])
    })

# 11. Save
with open("preproc_v2.json","w") as f: json.dump(to_py(preproc_manifest),f,indent=2)
with open("knn_model_v2.json","w") as f: json.dump(to_py(knn_export),f,indent=2)
with open("model_card_v2.md","w") as f: f.write(model_card)
with open("golden_examples_v2.json","w") as f: json.dump(to_py(golden),f,indent=2)

print("\nArtifacts written: preproc_v2.json, knn_model_v2.json, model_card_v2.md, golden_examples_v2.json")
print(f"\nTest Metrics:")
print(f"  ROC-AUC: {metrics['roc_auc']:.3f}")
print(f"  PR-AUC: {metrics['pr_auc']:.3f}")
print(f"  Precision: {metrics['precision']:.3f}")
print(f"  Recall: {metrics['recall']:.3f}")
print(f"  F1-Score: {metrics['f1']:.3f}")
print(f"  Specificity: {metrics['specificity']:.3f}")
print(f"  Threshold: {chosen_threshold:.3f}")

Starting hyperparameter tuning...
Fitting 5 folds for each of 30 candidates, totalling 150 fits

Best parameters: {'knn__algorithm': 'auto', 'knn__metric': 'euclidean', 'knn__n_neighbors': 3, 'knn__p': 1, 'knn__weights': 'distance'}
Best CV recall score: 0.268

Artifacts written: preproc_v2.json, knn_model_v2.json, model_card_v2.md, golden_examples_v2.json

Test Metrics:
  ROC-AUC: 0.685
  PR-AUC: 0.284
  Precision: 0.290
  Recall: 0.626
  F1-Score: 0.396
  Specificity: 0.714
  Threshold: 0.000
